<a href="https://colab.research.google.com/github/Monika171/Deep-Learning-with-Pytorch/blob/main/02_insurance_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1gy4dCMQaBlfthqAjyZuc3Dh8VlpwS_OC')

     |████████████████████████████████| 71kB 4.4MB/s 


# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [ ]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [2]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [3]:
project_name='02-insurance-linear-regression-monika171' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [4]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [5]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [6]:
your_name = 'monika' # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [7]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [8]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
1063,36,male,31.74045,3,no,7203.014555
1317,18,male,58.97430,0,no,1279.808970
140,34,male,24.88620,2,no,30113.495258
106,19,female,31.52400,1,no,2564.670900
634,51,male,44.06700,1,no,10330.480600


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [9]:
num_rows = dataframe.shape[0]
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [10]:
num_cols = dataframe.shape[1]
print(num_cols)

6


**Q: What are the column titles of the input variables?**

In [11]:
input_cols = [i for i in dataframe.columns[:-1]]
input_cols

['age', 'sex', 'bmi', 'children', 'smoker']

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [12]:
categorical_cols = [i for i in dataframe.select_dtypes(include=['object', 'category']).columns]
categorical_cols

['sex', 'smoker']

**Q: What are the column titles of output/target variable(s)?**

In [13]:
output_cols = [dataframe.columns[-1]]
output_cols

['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [ ]:
# Write your answer here

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [14]:
!pip install jovian --upgrade -q

In [15]:
import jovian

In [16]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/monika171/02-insurance-linear-regression-monika171


'https://jovian.ai/monika171/02-insurance-linear-regression-monika171'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [17]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [18]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[36.     ,  1.     , 31.74045,  3.     ,  0.     ],
        [18.     ,  1.     , 58.9743 ,  0.     ,  0.     ],
        [34.     ,  1.     , 24.8862 ,  2.     ,  0.     ],
        ...,
        [43.     ,  0.     , 39.6492 ,  2.     ,  0.     ],
        [49.     ,  1.     , 39.8046 ,  0.     ,  0.     ],
        [40.     ,  0.     , 32.856  ,  0.     ,  0.     ]]),
 array([[ 7203.014555],
        [ 1279.80897 ],
        [30113.495258],
        ...,
        [21059.034172],
        [ 8936.84924 ],
        [ 6502.0384  ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [19]:
inputs = torch.tensor(inputs_array, dtype=torch.float32)
targets = torch.tensor(targets_array, dtype=torch.float32)

In [20]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [21]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [62]:
val_percent = 0.18 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [63]:
batch_size = 6

In [64]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [65]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[18.0000,  0.0000, 40.9035,  0.0000,  0.0000],
        [22.0000,  0.0000, 26.9730,  0.0000,  0.0000],
        [48.0000,  1.0000, 40.7037,  1.0000,  0.0000],
        [30.0000,  1.0000, 43.1013,  1.0000,  0.0000],
        [58.0000,  1.0000, 39.6270,  0.0000,  0.0000],
        [54.0000,  0.0000, 52.6251,  0.0000,  1.0000]])
targets: tensor([[ 1792.8169],
        [ 2365.5159],
        [31315.8105],
        [20859.4883],
        [12499.0303],
        [70147.4688]])


Let's save our work by committing to Jovian.

In [66]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/monika171/02-insurance-linear-regression-monika171


'https://jovian.ai/monika171/02-insurance-linear-regression-monika171'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [67]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [68]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                          # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [69]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [70]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3680, -0.0414,  0.2695, -0.1386, -0.0510]], requires_grad=True),
 Parameter containing:
 tensor([0.2170], requires_grad=True)]

One final commit before we train the model.

In [71]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/monika171/02-insurance-linear-regression-monika171


'https://jovian.ai/monika171/02-insurance-linear-regression-monika171'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [72]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [73]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 14009.11328125}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [74]:
epochs = 1500
lr = 1e-2
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 7158.4438
Epoch [40], val_loss: 6964.0483
Epoch [60], val_loss: 6949.3765
Epoch [80], val_loss: 6948.9341
Epoch [100], val_loss: 6945.6812
Epoch [120], val_loss: 6937.0059
Epoch [140], val_loss: 6936.5117
Epoch [160], val_loss: 6934.9336
Epoch [180], val_loss: 6925.9014
Epoch [200], val_loss: 6923.1069
Epoch [220], val_loss: 6919.6133
Epoch [240], val_loss: 6916.3340
Epoch [260], val_loss: 6920.6646
Epoch [280], val_loss: 6916.8477
Epoch [300], val_loss: 6901.9312
Epoch [320], val_loss: 6904.8584
Epoch [340], val_loss: 6896.8765
Epoch [360], val_loss: 6897.7617
Epoch [380], val_loss: 6893.9741
Epoch [400], val_loss: 6895.6250
Epoch [420], val_loss: 6886.6572
Epoch [440], val_loss: 6886.5488
Epoch [460], val_loss: 6878.4121
Epoch [480], val_loss: 6879.9136
Epoch [500], val_loss: 6876.3604
Epoch [520], val_loss: 6878.7759
Epoch [540], val_loss: 6872.7207
Epoch [560], val_loss: 6876.6357
Epoch [580], val_loss: 6864.9233
Epoch [600], val_loss: 6872.5713
Epoch [620], v

In [75]:
epochs = 1500
lr = 1e-3
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6786.7930
Epoch [40], val_loss: 6786.1875
Epoch [60], val_loss: 6786.3813
Epoch [80], val_loss: 6785.9697
Epoch [100], val_loss: 6785.7407
Epoch [120], val_loss: 6785.8916
Epoch [140], val_loss: 6785.2715
Epoch [160], val_loss: 6785.5493
Epoch [180], val_loss: 6785.2627
Epoch [200], val_loss: 6785.0811
Epoch [220], val_loss: 6785.1870
Epoch [240], val_loss: 6784.8765
Epoch [260], val_loss: 6784.2422
Epoch [280], val_loss: 6784.2866
Epoch [300], val_loss: 6784.3662
Epoch [320], val_loss: 6784.2358
Epoch [340], val_loss: 6783.8589
Epoch [360], val_loss: 6783.4604
Epoch [380], val_loss: 6783.6782
Epoch [400], val_loss: 6783.4478
Epoch [420], val_loss: 6783.2983
Epoch [440], val_loss: 6783.5117
Epoch [460], val_loss: 6782.8345
Epoch [480], val_loss: 6782.4233
Epoch [500], val_loss: 6782.6094
Epoch [520], val_loss: 6782.6733
Epoch [540], val_loss: 6781.9980
Epoch [560], val_loss: 6781.8257
Epoch [580], val_loss: 6781.9126
Epoch [600], val_loss: 6781.4639
Epoch [620], v

In [76]:
epochs = 1500
lr = 1e-4
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6773.6621
Epoch [40], val_loss: 6773.6538
Epoch [60], val_loss: 6773.6475
Epoch [80], val_loss: 6773.6392
Epoch [100], val_loss: 6773.6167
Epoch [120], val_loss: 6773.6108
Epoch [140], val_loss: 6773.5879
Epoch [160], val_loss: 6773.5708
Epoch [180], val_loss: 6773.5659
Epoch [200], val_loss: 6773.5220
Epoch [220], val_loss: 6773.5161
Epoch [240], val_loss: 6773.4810
Epoch [260], val_loss: 6773.4751
Epoch [280], val_loss: 6773.4561
Epoch [300], val_loss: 6773.4458
Epoch [320], val_loss: 6773.4429
Epoch [340], val_loss: 6773.4014
Epoch [360], val_loss: 6773.3916
Epoch [380], val_loss: 6773.3804
Epoch [400], val_loss: 6773.3608
Epoch [420], val_loss: 6773.3354
Epoch [440], val_loss: 6773.3247
Epoch [460], val_loss: 6773.3232
Epoch [480], val_loss: 6773.2808
Epoch [500], val_loss: 6773.2300
Epoch [520], val_loss: 6773.2231
Epoch [540], val_loss: 6773.2246
Epoch [560], val_loss: 6773.2056
Epoch [580], val_loss: 6773.1968
Epoch [600], val_loss: 6773.1758
Epoch [620], v

In [77]:
epochs = 1500
lr = 1e-5
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6772.3564
Epoch [40], val_loss: 6772.3677
Epoch [60], val_loss: 6772.3735
Epoch [80], val_loss: 6772.3779
Epoch [100], val_loss: 6772.3809
Epoch [120], val_loss: 6772.3828
Epoch [140], val_loss: 6772.3838
Epoch [160], val_loss: 6772.3862
Epoch [180], val_loss: 6772.3843
Epoch [200], val_loss: 6772.3853
Epoch [220], val_loss: 6772.3833
Epoch [240], val_loss: 6772.3823
Epoch [260], val_loss: 6772.3838
Epoch [280], val_loss: 6772.3867
Epoch [300], val_loss: 6772.3853
Epoch [320], val_loss: 6772.3843
Epoch [340], val_loss: 6772.3857
Epoch [360], val_loss: 6772.3809
Epoch [380], val_loss: 6772.3818
Epoch [400], val_loss: 6772.3833
Epoch [420], val_loss: 6772.3833
Epoch [440], val_loss: 6772.3838
Epoch [460], val_loss: 6772.3862
Epoch [480], val_loss: 6772.3828
Epoch [500], val_loss: 6772.3853
Epoch [520], val_loss: 6772.3872
Epoch [540], val_loss: 6772.3867
Epoch [560], val_loss: 6772.3857
Epoch [580], val_loss: 6772.3843
Epoch [600], val_loss: 6772.3809
Epoch [620], v

In [78]:
epochs = 1500
lr = 1e-6
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6772.3823
Epoch [40], val_loss: 6772.3838
Epoch [60], val_loss: 6772.3838
Epoch [80], val_loss: 6772.3857
Epoch [100], val_loss: 6772.3843
Epoch [120], val_loss: 6772.3833
Epoch [140], val_loss: 6772.3843
Epoch [160], val_loss: 6772.3867
Epoch [180], val_loss: 6772.3911
Epoch [200], val_loss: 6772.3896
Epoch [220], val_loss: 6772.3882
Epoch [240], val_loss: 6772.3887
Epoch [260], val_loss: 6772.3867
Epoch [280], val_loss: 6772.3892
Epoch [300], val_loss: 6772.3921
Epoch [320], val_loss: 6772.3931
Epoch [340], val_loss: 6772.3936
Epoch [360], val_loss: 6772.3940
Epoch [380], val_loss: 6772.3965
Epoch [400], val_loss: 6772.3975
Epoch [420], val_loss: 6772.3975
Epoch [440], val_loss: 6772.3979
Epoch [460], val_loss: 6772.3984
Epoch [480], val_loss: 6772.4004
Epoch [500], val_loss: 6772.4023
Epoch [520], val_loss: 6772.4009
Epoch [540], val_loss: 6772.3970
Epoch [560], val_loss: 6772.3984
Epoch [580], val_loss: 6772.3984
Epoch [600], val_loss: 6772.3999
Epoch [620], v

**Q: What is the final validation loss of your model?**

In [79]:
val_loss = 6772.4282

Let's log the final validation loss to Jovian and commit the notebook

In [80]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [81]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/monika171/02-insurance-linear-regression-monika171


'https://jovian.ai/monika171/02-insurance-linear-regression-monika171'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [82]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(input)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [83]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([47.0000,  0.0000, 26.1960,  1.0000,  0.0000])
Target: tensor([9393.6377])
Prediction: tensor(11142.8604)


In [84]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([32.0000,  0.0000, 26.2515,  1.0000,  0.0000])
Target: tensor([19388.8633])
Prediction: tensor(6944.7847)


In [85]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([20.0000,  0.0000, 35.4312,  0.0000,  0.0000])
Target: tensor([2487.7256])
Prediction: tensor(2467.2742)


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [86]:
jovian.commit(project=project_name, environment=None)
# jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/monika171/02-insurance-linear-regression-monika171


'https://jovian.ai/monika171/02-insurance-linear-regression-monika171'